In [87]:
import numpy as np
import pandas as pd
import random
import re
from itertools import permutations,combinations
import nltk
from collections import defaultdict

In [1]:
class NGramSuggestion():
    @classmethod
    def initfromrawtext(cls,Ngram_size, rawtext_in,min_freq = 2):
        sents = rawtext_in[:-1].split('.')
        r_sents = []
        for sent in sents:
            new_sent = sent.strip()
            new_sent = new_sent.split()
            new_sent = ['<s>'] + new_sent + ['<\s>']
            r_sents += [new_sent]
        
        return cls(Ngram_size,r_sents,min_freq=2)

    def __init__(self,Ngram_size,text_L,min_freq=2):
        self.sents = text_L
        self.N = Ngram_size
        self.vocab = self.get_vocab(min_freq=2)
        self.v_len = len(self.vocab)
        self.counts = defaultdict(int)
        self.probs = dict()
        self.suggestions = dict()
        self.replace_unk()
        print(self.sents)
        #count N-grams from text sentences
        for sent in self.sents:
            for i in range(len(sent) - self.N + 1 ):
                for i_end in range(i+1,i+self.N+1):
                    self.counts[tuple(sent[i:i_end])] += 1
            for i in range(len(sent) - self.N + 1, len(sent)-1 ):
                    for i_end in range(len(sent) - self.N + 2,len(sent)+1):
                        self.counts[tuple(sent[i:i_end])] += 1

        

        
            
    def replace_unk(self):
        self.sents = [[x if x in self.vocab else '<unk>' for x in y] for y in self.sents]


    def suggest(self,text_L):
        less1gram = tuple([x if x in self.vocab else '<unk>' for x in text_L])
        gram_level = self.N
        while(gram_level > 2):
            if less1gram in self.counts:
                break
            less1gram = less1gram[1:]
            gram_level -= 1

        if less1gram in self.suggestions.keys():
            return self.suggestions[less1gram]
        
        bestword = ''
        bestprob = 0
        for word in self.vocab:
            prob = (self.counts[less1gram + (word,)] + 1)/(self.counts[less1gram] + self.v_len)
            if prob > bestprob:
                bestprob = prob
                bestword = word

        self.suggestions[less1gram] = bestword
        return bestword


    def get_counts(self):
        return self.counts


    @classmethod
    def suggestfromraw(cls,rawtext):
        text_L = [x.trim() for x in rawtext.split()]
        return cls.suggest(text_L)


    def get_vocab(self,min_freq=2):
        v = defaultdict(int)
        for sent in self.sents:
            for word in sent:
                v[word] += 1
        delkeys = []
        for key,val in v.items():
            if val < min_freq:
                delkeys += [key]

        for key in delkeys:
            del v[key]
        return set(v.keys())
